In [183]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [184]:
allFacilities = pd.read_csv("output/describeNetwork.csv", dtype={'primary_naics': object})
naics = pd.read_csv("data/naics.csv", dtype={'2012 NAICS US Code': object})

In [185]:
tri2013 = pd.read_csv('data/toxic-release-inventory.ny.2013.geoid.csv')


In [186]:
allChems = sorted(pd.unique(tri2013['chemical'].values.ravel()))
len(allChems)

181

In [187]:
allComms = sorted(pd.unique(allFacilities['Community'].values.ravel()))
allComms

[0, 1, 2, 3, 4, 5]

In [188]:
# chem = 'DIOXIN AND DIOXIN-LIKE COMPOUNDS'

In [189]:
naics = naics.drop(naics.columns[[0,3,4]], axis=1)
naics.columns = ["naics", "industry"]
allFacilities = allFacilities.merge(naics, left_on=["primary_naics"], right_on="naics", how="left")
allFacilities = allFacilities.drop(['Unnamed: 0', 'naics', 'Betweeness','Closeness','Eigenvector'], axis=1)

In [190]:
allFacilities

,Facility,Degrees,Clustering Coefficient,tri_facility_id,primary_naics,parent_company_name,Chemicals,Community,industry
0,IBM CORP,236,0.264443,12602BMCRPSOUTH,334111,IBM CORP,"['AMMONIA', 'COPPER COMPOUNDS', 'ETHYLENE GLYC...",0,Electronic Computer Manufacturing
1,IBM CORP,236,0.264443,12533BM EASTF,334413,IBM CORP,"['AMMONIA', 'COPPER COMPOUNDS', 'ETHYLENE GLYC...",0,Semiconductor and Related Device Manufacturing
2,EASTMAN KODAK CO EASTMAN BUSINESS PARK,228,0.284218,14652STMNK1669L,325992,EASTMAN KODAK CO,"['ACETONITRILE', 'AMMONIA', 'ANILINE', 'BARIUM...",0,"Photographic Film, Paper, Plate, and Chemical ..."
3,NORLITE LLC,225,0.243294,12047NRLTC628SO,327992,TRADEBE ENVIRONMENTAL SERVICES LLC,"['1,1,1-TRICHLOROETHANE', '1,1,2,2-TETRACHLORO...",1,Ground or Treated Mineral and Earth Manufactur...
4,ALCOA INC,193,0.332038,13662LMNMCPARKA,331313,ALCOA INC,"['BENZO(G,H,I)PERYLENE', 'CARBONYL SULFIDE', '...",4,Alumina Refining and Primary Aluminum Production
5,BUCKEYE TERMINALS LLC - RENSSELAER TERMINAL,188,0.399932,12144MRDHS367AM,493190,BUCKEYE TERMINALS LLC,"['1,2,4-TRIMETHYLBENZENE', 'BENZENE', 'BENZO(G...",1,Other Warehousing and Storage
6,MPM SILICONES LLC,179,0.322453,12188GNRLL260HU,325211,MOMENTIVE PERFORMANCE MATERIALS HOLDINGS LLC,"['AMMONIA', 'CHLORINE', 'CHLOROETHANE', 'CHLOR...",0,Plastics Material and Resin Manufacturing
7,INTERNATIONAL PAPER,178,0.336380,12883NTRNTSHORE,322121,INTERNATIONAL PAPER CO,"['ACETALDEHYDE', 'AMMONIA', 'BENZO(G,H,I)PERYL...",0,Paper (except Newsprint) Mills
8,NORTHVILLE INDUSTRIES CORP - SETAUKET TERMINAL,167,0.437126,11733NRTHV19TER,424710,NIC HOLDING CORP,"['1,2,4-TRIMETHYLBENZENE', 'BENZENE', 'CYCLOHE...",1,Petroleum Bulk Stations and Terminals
9,FINCH PAPER LLC,164,0.378498,12801FNCHP1GLEN,322121,FINCH PAPER HOLDINGS,"['AMMONIA', 'BENZO(G,H,I)PERYLENE', 'CHLORINE'...",0,Paper (except Newsprint) Mills


In [269]:
# cols = ['nonReportChem', 'Community', 'Facility', 'tri_facility_id', 'parent_company_name', 'industry', 'primary_naics', 'num_reporting']
cols = ['nonReportChem', 'Community', 'Facility', 'primary_naics', 'industry', 'num_reporting']

In [270]:
def chemPickCommunity(comm, chem):
    # Checks if chemical is emitted by node. If true assign node to withChem dataframe
    withChem = allFacilities[allFacilities['Chemicals'].str.contains(chem)]
    # If false, assign node to withoutChem
    withoutChem = allFacilities[~allFacilities['Chemicals'].str.contains(chem)]
    # Subset dataframe based on community
    withChemComm = withChem[withChem['Community'] == comm]
    withoutChemComm = withoutChem[withoutChem['Community'] == comm]
    # Get NAICS for all reporting facilities
    chemNaics = pd.unique(withChemComm['primary_naics'].values.ravel())
    # Check if a non-reporter has a counterpart that is reporting based on NAICS, community, and chemical
    nonReporters = withoutChemComm[withoutChemComm['primary_naics'].isin(chemNaics)]
    nonReporters['nonReportChem'] = chem

    if len(nonReporters) > 0:
        num = nonReporters.merge(withChemComm, on='primary_naics').groupby('tri_facility_id_x').size().order(ascending=False)
        num = num.reset_index()
        num = num.rename(columns = {0:'num_reporting', 'tri_facility_id_x': 'tri_facility_id'})
        nonReporters = nonReporters.merge(num, on='tri_facility_id')
        return (nonReporters[cols])


In [271]:
# chemPickCommunity(allComms[1], allChems[2])

In [272]:
allNonRepList = []

In [273]:
for i in allComms:
    for j in allChems:
        # Feeds a community value and chemical value to function
        allNonRepList.append(chemPickCommunity(i,j))

/Users/afair/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/afair/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:16: FutureWarning: order is deprecated, use sort_values(...)
/Users/afair/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  app.launch_new_instance()
/Users/afair/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [274]:
final = pd.concat(i for i in allNonRepList)
final = final.sort(['Community', 'nonReportChem', 'num_reporting'], ascending=[True, True, False])

/Users/afair/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from IPython.kernel.zmq import kernelapp as app


In [275]:
final.to_csv('output/nonReporters.csv')

In [276]:
final[final['nonReportChem']=='BENZENE']

,nonReportChem,Community,Facility,primary_naics,industry,num_reporting
0,BENZENE,1,GRIFFITH ENERGY - PALMYRA,424710,Petroleum Bulk Stations and Terminals,22
1,BENZENE,1,GRIFFITH ENERGY - WATERTOWN,424710,Petroleum Bulk Stations and Terminals,22
2,BENZENE,1,GRIFFITH ENERGY - LODI,424710,Petroleum Bulk Stations and Terminals,22
7,BENZENE,1,COMMANDER TERMINALS LLC,424710,Petroleum Bulk Stations and Terminals,22
8,BENZENE,1,CCI ROSETON LLC,221112,Fossil Fuel Electric Power Generation,3
3,BENZENE,1,SABIC INNOVATIVE PLASTICS US LLC,325211,Plastics Material and Resin Manufacturing,2
4,BENZENE,1,CYTEC OLEAN INC,325211,Plastics Material and Resin Manufacturing,2
9,BENZENE,1,3M CO - TONAWANDA,325211,Plastics Material and Resin Manufacturing,2
5,BENZENE,1,AVERY DENNISON INFORMATION & BRAND MANAGEMENT DIV,313320,Fabric Coating Mills,1
6,BENZENE,1,ADCHEM CORP,322220,Paper Bag and Coated and Treated Paper Manufac...,1
